# 智能问答机器人：简历筛选小助手

## 目标
通过对本地简历库的简历进行解析、切片、创建索引，实现基于JD进行简历筛选，并对筛选的Top1简历进行总结的功能

## 准备工作
### 平台注册
1.先在appbuilder平台注册，获取token
2.创建BES集群，详见(https://cloud.baidu.com/doc/BES/s/0jwvyk4tv)
3.安装appbuilder-sdk

In [ ]:
!pip install appbuilder-sdk
!pip install elasticsearch==7.11.0

In [1]:
import logging
import os

#  设置环境变量
os.environ["APPBUILDER_TOKEN"] = "..."

# 创建BES集群后获得集群id、用户名、密码
cluster_id = "..."
username = "..."
password = "..."

print("init done")


init done


## 开发过程
### 对简历文档内容进行解析、切分

In [ ]:
import appbuilder
from pathlib import Path

# 基于doc_parser和doc_splitter解析file_path文件为若干个段落
def parse_file(file_path, doc_parser, doc_splitter):
    input_msg = appbuilder.Message(str(file_path))
    doc_parser_result = doc_parser(input_msg, return_raw=True)
    doc_splitter_result = doc_splitter(doc_parser_result)
    return [f"{file_path.name}+{para['text'][:384]}" 
            for para in doc_splitter_result.content["paragraphs"]]

# 文档切分的分块大小，每个分块最大340个字符
chunk_size = 340

# 本地简历库地址
file_dir = "../简历"

# 声明文档解析和文档切分组件
doc_parser = appbuilder.DocParser()
doc_splitter = appbuilder.DocSplitter(splitter_type="split_by_chunk", max_segment_length=chunk_size)       

# 批量解析，形成段落切片列表
resume_paragraphs = [para_text 
            for file in Path(file_dir).iterdir() if file.is_file()
            for para_text in parse_file(file, doc_parser, doc_splitter)]

print(f"total length of splitted paragraphs {len(resume_paragraphs)}")

print("sample resume paragraphs")
for paras in resume_paragraphs[:3]:
    print(paras)

### 计算文档切片的语义向量并入库

In [ ]:
import time
# 千帆Embedding
embedding = appbuilder.Embedding()

# 将段落切片列表入库到BESVectorStoreIndex，这里面用到的Baidu Elastic Search服务
segments = appbuilder.Message(resume_paragraphs)
vector_index = appbuilder.BESVectorStoreIndex.from_segments(
    segments=segments, cluster_id=cluster_id, user_name=username, 
    password=password, embedding=embedding)

# bes向量入库是异步操作，此处等待bes进行refresh
time.sleep(5)
print("index done.")

### 从工作职责描述中抽取关键词

In [ ]:
# 招聘jd
from collections import Counter, defaultdict

# 工作职责描述
job_desc = '''拥有大语言模型领域的研究背景，发表过期刊会议论文者加分；
具备深度学习平台开发或大模型开发经验，能够独立完成任务；
具备优秀的编程能力，熟悉tensorflow、pytorch、paddlepaddle中的一种深度学习框架；
对人工智能有浓厚兴趣，愿意投入时间和精力深入研究；
具备团队协作精神，能够与团队成员有效沟通，共同推进项目进展。'''

# 标签抽取的组件
tagger = appbuilder.TagExtraction(model="ERNIE Speed-AppBuilder")

# 从JD抽取标签并打印
tags = tagger(appbuilder.Message(job_desc))

print("从JD抽取标签并打印")
print(tags.content)
tag_list = [tag.split(".")[1] for tag in tags.content.split("\n")]


### 基于工作描述的关键词从简历库中检索简历并排序

In [ ]:
# 基于JD抽取出来的标签，使用retriever对简历库进行检索，基于简历文件名称命中次数进行汇总和排序
retriever = vector_index.as_retriever()

resume_count = Counter()
resume_content = defaultdict(set)

for tag in tag_list:
    print("Going to retrieve resumes for tag: ", tag, "\nGot Results")
    relevant_resumes = retriever(query=appbuilder.Message(tag), top_k=3)
    for idx, res in enumerate(relevant_resumes.content):
        name_and_content = res["text"].split("+")
        print(f"{idx+1}: {name_and_content[0]}, {name_and_content[1]}")
        resume_count[name_and_content[0]] += 1
        resume_content[name_and_content[0]].add(name_and_content[1])
    print()

sorted_resumes = sorted(resume_count.items(), key=lambda x: x[1], reverse=True)
print(sorted_resumes)

### 对检索得到的最优简历进行总结

In [ ]:

play = appbuilder.Playground(
    prompt_template="基于候选人姓名、职责描述和简历内容，概括一下{name}的推荐理由。\n候选人姓名: {name}\n职责描述: {JD}\n简历内容: {resume}\n推荐理由: ",
    model="ERNIE Speed-AppBuilder"
)

resume_summary = play(appbuilder.Message({"JD": job_desc, "name": sorted_resumes[0][0], "resume": "\n".join(list(resume_content[sorted_resumes[0][0]]))}))
print(resume_summary.content)